In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

In [2]:
data  = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data.head()
Col = list(data.columns)
Col.remove('Personal Loan')
Col.append('Personal Loan')
data = data[Col].drop(columns='ZIP Code')

In [3]:
from sklearn.model_selection import train_test_split
X = data.values[:,:-1]
X = X.reshape(X.shape[0],1,X.shape[1])
Y = data.values[:,-1].reshape(X.shape[0],1)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.25)

In [4]:
def Sigmoid(x):
    s = 1 / (1+np.exp(-1*x))
    return s

In [5]:
def dri_Sigmoid(x):
    s = Sigmoid(x) * (1-Sigmoid(x))
    return s

In [6]:
def Predict(S,x_test,y_test):
    from sklearn.metrics import accuracy_score
    wI = S['w_inp']
    wH = S['w_hid']
    inp_out = (wI*x_test)
    inp_out = inp_out.reshape(inp_out.shape[0],4,3).sum(axis=2)
    inp_out = Sigmoid(inp_out)
    hid_out = wH*inp_out
    hid_out = hid_out.reshape(hid_out.shape[0],1,4).sum(axis=2)
    hid_out = np.floor(Sigmoid(hid_out))

    
    return accuracy_score(hid_out,y_test)*100

In [7]:
w_inp = np.random.rand(1,12)
w_hid = np.random.rand(1,4)
Belif_Space =[]

In [8]:
#Generating Population
pop_no_sol = 100
w_pop=[]
for i in range(pop_no_sol):
    W_inp = np.random.rand(1,12)
    W_hid = np.random.rand(1,4)
    temp = np.append(W_inp,W_hid,axis=1)
    w_pop.append(temp)
w_pop.append(np.append(w_inp,w_hid,axis=1))
w_pop = np.array(w_pop)

In [9]:
def D1toMat(sol):
    Z = {
        "w_inp" : (sol.reshape(4,4)[:3]).reshape(1,12),
        "w_hid" : sol.reshape(4,4)[-1]
    }
    return Z

In [10]:
def Fitness(sol,X,Y):
    fitness =[]
    for i in sol :
        Z = D1toMat(i)
        fitness.append(Predict(Z,X,Y))

    return fitness

In [11]:
def Belif_Space(DF):
    return DF.sort_values(by='Fitness',ascending=False)['Weights'].values[:10]

In [12]:
DF = pd.DataFrame()
DF['Weights'] = list(w_pop)

In [13]:
DF['Fitness'] = Fitness(list(DF['Weights'].values),X,Y) 
Belif_Space = Belif_Space(DF)

In [14]:
def CrossOver(par1,par2):
    if(isinstance(par2,int)):
        offsp1 = np.append(DF['Weights'][par1][0][:8],DF['Weights'][par2][0][8:]).reshape(1,16)
        offsp2 = np.append(DF['Weights'][par2][0][:8],DF['Weights'][par1][0][8:]).reshape(1,16)
    else:
        offsp1 = np.append(DF['Weights'][par1][0][:8],par2[0][8:]).reshape(1,16)
        offsp2 = np.append(par2[0][:8],DF['Weights'][par1][0][8:]).reshape(1,16)
    return offsp1,offsp2

In [15]:
def Influence(Belif):
    pos1 = np.random.randint(0,Belif.shape[0])
    pos2 = np.random.randint(0,Belif.shape[0])
    while(pos1==pos2):
        pos2 = np.random.randint(0,Belif.shape[0])
    inf1 = Belif[pos1]
    inf2 = Belif[pos2]
    return inf1,inf2

In [16]:
def Select(size):
    p1 = np.random.randint(0,size)
    p2 = np.random.randint(0,size)
    while(p2==p1):
        p2 = np.random.randint(0,size)
    return p1,p2

In [17]:
def Mutate(offsp1,offsp2):
    pos1 = np.random.randint(0,16)
    pos2 = np.random.randint(0,16)
    while(pos1==pos2):
        pos2 = np.random.randint(0,16)
    offsp1[0][pos1],offsp1[0][pos2] = offsp1[0][pos2],offsp1[0][pos1]
    offsp2[0][pos1],offsp2[0][pos2] = offsp2[0][pos2],offsp2[0][pos1]
    return offsp1,offsp2

In [18]:
def Change(DF,ofs,p1,p2,x_test,y_test):
    warnings.filterwarnings('ignore')
    fitness = Fitness(ofs,x_test,y_test)
    DF.loc[p1,['Weights','Fitness']]  = [ofs[0],fitness[0]]
    DF.loc[p2,['Weights','Fitness']] = [ofs[1],fitness[1]]

In [27]:
def Choose(d):
    import random
    ch1 = random.randint(0,len(d)-1)
    ch2 = random.randint(0,len(d)-1)
    while(ch1==ch2):
        ch2 = random.randint(0,len(d)-1)
    return d[ch1],d[ch2]

In [28]:

def Cultural_Algo(DF,Belif,X,Y,Epoch=50):
    for i in tqdm(range(Epoch)):
        #Fitness
        DF['Fitness'] = Fitness(list(DF['Weights'].values),X,Y) 
        #selection 
        p1,p2 = Select(DF['Weights'].shape[0])
        # Influence Selection
        inf1,inf2 = Influence(Belif)
        #Crossover
        o1,o2 = CrossOver(p1,inf1)
        o3,o4 = CrossOver(p2,inf2)
        #Mutation 
        o1,o2 = Mutate(o1,o2)
        o3,o4 = Mutate(o3,o4)
        d = [o1,o2,o3,o4]
        ch1,ch2 = Choose(d)
        ofs = np.stack((ch1,ch2))
        #Check
        Change(DF,ofs,p1,p2,X,Y)
    DF.sort_values(by='Fitness',ascending=False)
    Best = D1toMat(DF['Weights'][0])
    Fit = DF['Fitness'][0]
    return Best,Fit

In [29]:
Cultural_Algo(DF,Belif_Space,x_test,y_test,1000)

100%|██████████| 1000/1000 [01:04<00:00, 15.47it/s]


({'w_inp': array([[0.72417188, 0.90836672, 0.34640091, 0.53506874, 0.21010481,
          0.94473288, 0.88882908, 0.62736644, 0.15365727, 0.35334795,
          0.75144496, 0.68779149]]),
  'w_hid': array([0.91128709, 0.27677498, 0.93691671, 0.84296795])},
 92.4)

# END